# Part3: 倒排表扩展与优化

## 核心功能
- ✅ 位置信息存储：每个词项记录在文档中的具体位置
- ✅ 按块存储压缩：将连续词项分组存储，减少指针数量
- ✅ 前端编码压缩：利用词项共同前缀进行压缩
- ✅ 压缩效果分析：量化比较不同压缩方法的效果

## 工作流程

### 阶段1: 数据加载与位置信息构建
- **输入**: `inverted_index.json` + `normalized_tokens.json`
- **处理**: 从规范化词表重建位置信息
- **输出**: 带位置信息的倒排记录

### 阶段2: 跳表指针优化
- **算法**: √n间隔跳表指针
- **条件**: 长度≥4的倒排列表
- **效果**: 优化交集运算性能

### 阶段3: 压缩算法实现
- **按块存储**: 4个词项/块，减少指针数量
- **前端编码**: 利用词项共同前缀压缩

### 阶段4: 结果保存
- **输出文件**: `enhanced_inverted_index.json`
- **格式**: 四元组格式


In [ ]:
# 环境准备和导入
import json
import math
from collections import defaultdict
from typing import List, Tuple, Dict, Any, Set

# 运行
exec(open('Part3_Enhanced_Index.py').read())


In [ ]:
# 阶段1: 数据加载和基础信息展示
print("=" * 60)
print("阶段1: 数据加载与位置信息构建")
print("=" * 60)

# 创建索引实例
index = EnhancedInvertedIndex()

# 加载数据
print("加载基础倒排索引和规范化词表...")
index.load_from_basic_index("inverted_index.json", "normalized_tokens.json")

# 显示基础信息
print(f"词汇表大小: {len(index.vocabulary)} 个词项")
print(f"文档数量: {len(index.doc_lengths)} 个文档")
print(f"倒排列表数量: {len(index.inverted_lists)} 个")

# 显示词汇表示例
print("\n词汇表示例:")
for i, (term, term_id) in enumerate(list(index.vocabulary.items())[:5]):
    print(f"  {term}: {term_id}")

print("\n位置信息添加完成!")


In [ ]:
# 位置信息构建演示
print("\n" + "=" * 60)
print("位置信息构建演示")
print("=" * 60)

print("倒排记录格式: [文档ID, 词频, 位置列表, 跳表指针]")

# 显示几个示例
print("\n位置信息示例:")
for term_id, postings in list(index.inverted_lists.items())[:3]:
    term = [k for k, v in index.vocabulary.items() if v == int(term_id)][0]
    print(f"词项 '{term}' (ID: {term_id}):")
    for posting in postings[:2]:  # 只显示前2个
        print(f"  {posting}")
    print()


In [ ]:
# 阶段2: 跳表指针优化
print("=" * 60)
print("阶段2: 跳表指针优化")
print("=" * 60)

# 跳表指针优化
print("添加跳表指针...")
skip_count = 0
for term_id, postings in index.inverted_lists.items():
    if len(postings) >= 4:
        skip_count += index._add_skip_pointers(int(term_id), postings)

print(f"添加了 {skip_count} 个跳表指针")

# 显示跳表指针示例
print("\n跳表指针示例:")
for term_id, postings in list(index.inverted_lists.items()):
    if any(posting[3] != -1 for posting in postings):
        term = [k for k, v in index.vocabulary.items() if v == int(term_id)][0]
        print(f"词项 '{term}': {postings}")
        break


In [ ]:
# 阶段3: 压缩算法实现
print("\n" + "=" * 60)
print("阶段3: 压缩算法实现")
print("=" * 60)

# 按块存储压缩
print("实现按块存储压缩 (块大小: 4)...")
index.implement_blocking_compression()
print(f"按块存储完成: {len(index.compressed_lists)} 个块")

# 显示压缩块示例
print("\n压缩块示例:")
sample_block = list(index.compressed_lists.values())[0]
print(f"块内词项: {sample_block['terms']}")
print(f"词项ID: {sample_block['term_ids']}")
print(f"倒排记录: {sample_block['postings'][:2]}...")

# 前端编码压缩
print("\n实现前端编码压缩...")
index.implement_front_coding_compression()
print(f"前端编码完成: {len(index.front_coded_terms)} 个压缩条目")

# 显示前端编码示例
print("\n前端编码示例:")
for entry in index.front_coded_terms[:3]:
    if entry['type'] == 'prefix':
        print(f"前缀条目: '{entry['prefix']}' + {entry['suffixes']}")
    else:
        print(f"单个词项: '{entry['term']}'")


In [ ]:
# 压缩效果分析
print("\n" + "=" * 60)
print("压缩效果分析")
print("=" * 60)

# 压缩统计信息
print("压缩统计信息:")
index.print_compression_statistics()

# 详细分析
sizes = index.calculate_storage_sizes()
print(f"\n详细分析:")
print(f"原始格式: {sizes['original']:,} 字节")
if 'blocking' in sizes:
    ratio = sizes['blocking'] / sizes['original']
    print(f"按块存储: {sizes['blocking']:,} 字节 (压缩比: {ratio:.2%})")
if 'front_coding' in sizes:
    ratio = sizes['front_coding'] / sizes['original']
    print(f"前端编码: {sizes['front_coding']:,} 字节 (压缩比: {ratio:.2%})")


In [ ]:
# 阶段4: 保存结果和总结
print("\n" + "=" * 60)
print("阶段4: 保存结果")
print("=" * 60)

# 保存结果
print("保存增强倒排索引...")
index.save_enhanced_index("enhanced_inverted_index.json")
print("保存完成!")

# 总结
print("\n" + "="*60)
print("Part3 倒排表扩展与优化完成!")
print("="*60)
print("✅ 位置信息添加完成")
print("✅ 跳表指针优化完成") 
print("✅ 按块存储压缩完成")
print("✅ 前端编码压缩完成")
print("✅ 压缩效果分析完成")
print(f"✅ 结果已保存到: enhanced_inverted_index.json")
